# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#add g_key and make sure api_key file is in correct location

In [3]:
city_weather = pd.read_csv('../WeatherPy/city_weather.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
make sure g_key is in correct location and run from start

In [ ]:
# Plot heatmap
fig = gmaps.figure()

# Create heat layer
locations = city_weather["Lat", "Lng"]
weight = city_weather["Humidity"]
#may have to add .astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=rating)
#may add other heatmap settings such as dissipating=False, max_intensity=10, point_radius=1

# Add layer
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Drop null
city_weather.dropna(inplace=true)

# Drop based on weather criteria
drop_temps = city_weather["Max Temp] < 75
drop_temps2 = city_weather["Max Temp"] >=89
drop_hum  city_weather["Max Temp"]   < 85
drop_wind = city_weather["Wind Speed"] > 13
drop_cloud = city_weather["Cloudiness"] >=40

vac_cities = city_weather.drop(drop_temp, drop_temps2, drop_hum, drop_wind, drop_cloud)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Perfom API call 
base_url = "https://maps.googleapis.com/maps/api/staticmap?"

#what do I need special?
units = "imperial"

#define query_url
query_url = base_url + "&appid=" + weather_api_key + "&q=" + city + "&units=" + units


lats = []
lngs = []

In [ ]:
#heatmap layer example
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

#Example
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

# Display figure
